In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glucocorticoid_Sensitivity"
cohort = "GSE66705"

# Input paths
in_trait_dir = "../../input/GEO/Glucocorticoid_Sensitivity"
in_cohort_dir = "../../input/GEO/Glucocorticoid_Sensitivity/GSE66705"

# Output paths
out_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/GSE66705.csv"
out_gene_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE66705.csv"
out_clinical_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE66705.csv"
json_path = "../../output/preprocess/Glucocorticoid_Sensitivity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"NALP3 inflammasome up-regulation and CASP1 cleavage of the glucocorticoid receptor causes glucocorticoid resistance in leukemia cells [HG-U133_Plus_2]"
!Series_summary	"Glucocorticoids are universally used in the treatment of acute lymphoblastic leukemia (ALL), and glucocorticoid resistance in leukemia cells confers a poor prognosis.  To elucidate mechanisms of glucocorticoid resistance, we determined the prednisolone sensitivity of primary leukemia cells from 444 newly diagnosed ALL patients and found significantly higher expression of caspase 1 (CASP1) and its activator NLRP3 in glucocorticoid resistant leukemia cells, due to significantly lower somatic methylation of CASP1 and NLRP3 promoters.  Over-expression of CASP1 resulted in cleavage of the glucocorticoid receptor, diminished glucocorticoid-induced transcriptional response and increased glucocorticoid resistance. Knockdown or inhibition of CASP1 significantly increased glucocorticoid rece

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Looking at the background info, this dataset contains gene expression profiling data on HG-U133_Plus_2 platform
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# Trait: Glucocorticoid sensitivity appears to be available in the 'predlc50group' field (row 0)
# with values RES (resistant), SEN (sensitive), and INT (intermediate)
trait_row = 0

# Age: Not available in the sample characteristics dictionary
age_row = None

# Gender: Not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value):
    """
    Convert glucocorticoid sensitivity values to binary format.
    RES (resistant) = 1, SEN (sensitive) = 0, INT (intermediate) = None, #N/A = None
    """
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
        
    if value == "RES":
        return 1  # Resistant
    elif value == "SEN":
        return 0  # Sensitive
    elif value == "INT" or value == "#N/A":
        return None  # Intermediate or missing values
    else:
        return None

def convert_age(value):
    """Placeholder function for age conversion. Not used in this dataset."""
    return None

def convert_gender(value):
    """Placeholder function for gender conversion. Not used in this dataset."""
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,  # This variable comes from previous steps
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM1629982': [nan], 'GSM1629983': [nan], 'GSM1629984': [nan], 'GSM1629985': [nan], 'GSM1629986': [1.0], 'GSM1629987': [nan], 'GSM1629988': [1.0], 'GSM1629989': [1.0], 'GSM1629990': [nan], 'GSM1629991': [nan], 'GSM1629992': [nan], 'GSM1629993': [nan], 'GSM1629994': [nan], 'GSM1629995': [0.0], 'GSM1629996': [nan], 'GSM1629997': [nan], 'GSM1629998': [nan], 'GSM1629999': [nan], 'GSM1630000': [nan], 'GSM1630001': [0.0], 'GSM1630002': [1.0], 'GSM1630003': [nan], 'GSM1630004': [nan], 'GSM1630005': [nan], 'GSM1630006': [nan], 'GSM1630007': [nan], 'GSM1630008': [nan], 'GSM1630009': [nan], 'GSM1630010': [0.0], 'GSM1630011': [0.0], 'GSM1630012': [nan], 'GSM1630013': [nan], 'GSM1630014': [1.0], 'GSM1630015': [nan], 'GSM1630016': [0.0], 'GSM1630017': [nan], 'GSM1630018': [nan], 'GSM1630019': [nan], 'GSM1630020': [nan], 'GSM1630021': [0.0], 'GSM1630022': [nan], 'GSM1630023': [nan], 'GSM1630024': [1.0], 'GSM1630025': [0.0], 'GSM1630026': [1.0], 'GSM1630027': [0.0], 'GSM163002

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 68
Header line: "ID_REF"	"GSM1629982"	"GSM1629983"	"GSM1629984"	"GSM1629985"	"GSM1629986"	"GSM1629987"	"GSM1629988"	"GSM1629989"	"GSM1629990"	"GSM1629991"	"GSM1629992"	"GSM1629993"	"GSM1629994"	"GSM1629995"	"GSM1629996"	"GSM1629997"	"GSM1629998"	"GSM1629999"	"GSM1630000"	"GSM1630001"	"GSM1630002"	"GSM1630003"	"GSM1630004"	"GSM1630005"	"GSM1630006"	"GSM1630007"	"GSM1630008"	"GSM1630009"	"GSM1630010"	"GSM1630011"	"GSM1630012"	"GSM1630013"	"GSM1630014"	"GSM1630015"	"GSM1630016"	"GSM1630017"	"GSM1630018"	"GSM1630019"	"GSM1630020"	"GSM1630021"	"GSM1630022"	"GSM1630023"	"GSM1630024"	"GSM1630025"	"GSM1630026"	"GSM1630027"	"GSM1630028"	"GSM1630029"	"GSM1630030"	"GSM1630031"	"GSM1630032"	"GSM1630033"	"GSM1630034"	"GSM1630035"	"GSM1630036"	"GSM1630037"	"GSM1630038"	"GSM1630039"	"GSM1630040"	"GSM1630041"	"GSM1630042"	"GSM1630043"	"GSM1630044"	"GSM1630045"	"GSM1630046"	"GSM1630047"	"GSM1630048"	"GSM1630049"	"GSM1630050"	"GSM1630051"	"GSM1630052"	"GSM1630053"	"GSM1630054"	

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Let's examine the gene identifiers in the data
# These look like Affymetrix probe IDs (e.g., "1007_s_at"), which are microarray probe identifiers
# They are not human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE66705
Line 6: !Series_title = NALP3 inflammasome up-regulation and CASP1 cleavage of the glucocorticoid receptor causes glucocorticoid resistance in leukemia cells [HG-U133_Plus_2]
Line 7: !Series_geo_accession = GSE66705
Line 8: !Series_status = Public on Mar 24 2015
Line 9: !Series_submission_date = Mar 09 2015
Line 10: !Series_last_update_date = Sep 08 2020
Line 11: !Series_pubmed_id = 25938942
Line 12: !Series_pubmed_id = 32885175
Line 13: !Series_summary = Glucocorticoids are universally used in the treatment of acute lymphoblastic leukemia (ALL), and glucocorticoid resistance in leukemia cells confers a poor prognosis.  To elucidate mechanisms of glucocorticoid resistance, we determined the prednis


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DE

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which keys in gene annotation map to gene expression data identifiers
# From the output, we can see 'ID' in gene_annotation contains probe IDs (like "1007_s_at")
# and 'Gene Symbol' contains corresponding human gene symbols

# 2. Get gene mapping dataframe by extracting ID and Gene Symbol columns
gene_mapping = gene_annotation[['ID', 'Gene Symbol']].copy()
gene_mapping = gene_mapping.rename(columns={'Gene Symbol': 'Gene'})

# Show a sample of the mapping
print("Gene mapping sample (first 5 rows):")
print(gene_mapping.head())

# 3. Convert probe-level measurements to gene expression data
# First, load the gene expression data if not already loaded
genetic_data = get_genetic_data(matrix_file)

# Apply the gene mapping to convert probes to genes
gene_data = apply_gene_mapping(genetic_data, gene_mapping)

# Show a preview of the converted gene expression data
print("\nGene expression data preview (first 5 genes, first 5 samples):")
print(gene_data.iloc[:5, :5])

# Save the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


Gene mapping sample (first 5 rows):
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A



Gene expression data preview (first 5 genes, first 5 samples):
          GSM1629982  GSM1629983  GSM1629984  GSM1629985  GSM1629986
Gene                                                                
A1BG        8.062334    8.511603    8.369798    8.650215    8.574723
A1BG-AS1    4.396879    8.041974    6.771957    7.613696    7.900742
A1CF       12.404052   11.460827    7.749095   10.915274   10.909873
A2M        14.696204   11.655836   16.176442   15.562496   12.742218
A2M-AS1    10.296652    7.497209   10.200534   10.431418    8.051979



Gene expression data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE66705.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Clinical data shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.iloc[:, :5])  # Show first 5 columns only

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col != trait]
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing}")
    
    if len(linked_data) > 0:  # Ensure we have samples before checking
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 6. Final validation and save
note = "Dataset contains gene expression data from glucocorticoid sensitivity studies. "
note += "No demographic features available. " 

is_gene_available = len(normalized_gene_data) > 0
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=True, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (19845, 171)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Gene data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE66705.csv
Clinical data shape: (1, 171)
Clinical data preview:
                            GSM1629982  GSM1629983  GSM1629984  GSM1629985  \
Glucocorticoid_Sensitivity         NaN         NaN         NaN         NaN   

                            GSM1629986  
Glucocorticoid_Sensitivity         1.0  
Linked data shape: (171, 19846)
Linked data preview (first 5 rows, first 5 columns):
            Glucocorticoid_Sensitivity      A1BG  A1BG-AS1       A1CF  \
GSM1629982                         NaN  8.062334  4.396879  12.404052   
GSM1629983                         NaN  8.511603  8.041974  11.460827   
GSM1629984                         NaN  8.369798  6.771957   7.749095   
GSM1629985                         NaN  8.650215  7.613696  10.915274   
GSM1629986                         1.0  8.574723  7.900742  10.909873   

                  A2M  
GSM1629982  14.696204  
GSM1629983  11.655836  
GSM1629984  16.1764

Data shape after handling missing values: (84, 19846)
For the feature 'Glucocorticoid_Sensitivity', the least common label is '1.0' with 32 occurrences. This represents 38.10% of the dataset.
The distribution of the feature 'Glucocorticoid_Sensitivity' in this dataset is fine.



Linked data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/GSE66705.csv
